# Lab 2: Data Cleaning and Preparation
In this notebook, we clean and prepare the automobile dataset. We'll explore the data, handle missing values, normalize and transform data, visualize distributions, and create dummy variables for categorical data.

## Import necessary libraries

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

## Load dataset
We load the dataset and assign headers manually since no headers are present in the file.

In [ ]:
filename = Path.cwd().parent.parent/"Data"/"auto.csv"
headers = [
    "symboling", "normalized-losses", "make", "fuel-type", "aspiration",
    "num-of-doors", "body-style", "drive-wheels", "engine-location",
    "wheel-base", "length", "width", "height", "curb-weight",
    "engine-type", "num-of-cylinders", "engine-size", "fuel-system",
    "bore", "stroke", "compression-ratio", "horsepower",
    "peak-rpm", "city-mpg", "highway-mpg", "price"
]
df = pd.read_csv(filename, names=headers)
df.head(5)

## Replace '?' with NaN
We replace '?' with `np.nan` so pandas can recognize missing values.

In [ ]:
df.replace("?", np.nan, inplace=True)
df.head(5)

## Check for missing data

In [ ]:
missing_data = df.isnull()
missing_data.head(5)

In [ ]:
for column in missing_data.columns.values.tolist():
    print(f"Missing in {column}:")
    print(missing_data[column].value_counts())
    print("")

## Handle missing values

In [ ]:
df["normalized-losses"] = df["normalized-losses"].astype(float)
avg_norm_loss = df["normalized-losses"].mean()
print(f"Average Normalized Loss: {avg_norm_loss}")
df["normalized-losses"].replace(
    np.nan, avg_norm_loss, inplace=True
)

df["bore"] = df["bore"].astype(float)
avg_bore = df["bore"].mean()
print(f"Average Bore: {avg_bore}")
df["bore"].replace(np.nan, avg_bore, inplace=True)

df["stroke"] = df["stroke"].astype(float)
avg_stroke = df["stroke"].mean()
print(f"Average Stroke: {avg_stroke}")
df["stroke"].replace(np.nan, avg_stroke, inplace=True)

df["horsepower"] = df["horsepower"].astype(float)
avg_hp = df["horsepower"].mean()
print(f"Average Horsepower: {avg_hp}")
df["horsepower"].replace(np.nan, avg_hp, inplace=True)

df["peak-rpm"] = df["peak-rpm"].astype(float)
avg_rpm = df["peak-rpm"].mean()
print(f"Average Peak RPM: {avg_rpm}")
df["peak-rpm"].replace(np.nan, avg_rpm, inplace=True)

most_common_doors = df["num-of-doors"].value_counts().idxmax()
print(f"Most common number of doors: {most_common_doors}")
df["num-of-doors"].replace(np.nan, most_common_doors, inplace=True)

df.dropna(subset=["price"], inplace=True)
df.reset_index(drop=True, inplace=True)

## Convert data types

In [ ]:
df[["bore", "stroke", "peak-rpm", "price"]] = df[[
    "bore", "stroke", "peak-rpm", "price"
]].astype(float)
df["normalized-losses"] = df["normalized-losses"].astype(int)
df["horsepower"] = df["horsepower"].astype(int)
df.dtypes

## Convert mpg and normalize dimensions

In [ ]:
df["city-mpg"] = 235 / df["city-mpg"]
df.rename(columns={"city-mpg": "city-L/100km"}, inplace=True)
df["highway-mpg"] = 235 / df["highway-mpg"]
df.rename(columns={"highway-mpg": "highway-L/100km"}, inplace=True)
df["length"] = df["length"] / df["length"].max()
df["width"] = df["width"] / df["width"].max()
df["height"] = df["height"] / df["height"].max()
df.head()

## Visualize horsepower distribution

In [ ]:
plt.hist(df["horsepower"], bins=3)
plt.xlabel("horsepower")
plt.ylabel("count")
plt.title("Horsepower Distribution")
plt.show()

## Bin horsepower into categories

In [ ]:
bins = np.linspace(
    df["horsepower"].min(), df["horsepower"].max(), 4
)
group_names = ["Low", "Medium", "High"]
df["horsepower-binned"] = pd.cut(
    df["horsepower"], bins, labels=group_names, include_lowest=True
)
df[["horsepower", "horsepower-binned"]].head(10)

In [ ]:
df["horsepower-binned"].value_counts().sort_index()

In [ ]:
df["horsepower-binned"].value_counts().sort_index().plot(
    kind="bar"
)
plt.xlabel("Horsepower Bins")
plt.ylabel("Count")
plt.title("Horsepower Binned Distribution")
plt.show()

## Create dummy variables

In [ ]:
fuel_dummies = pd.get_dummies(
    df["fuel-type"]
).rename(columns={
    "diesel": "fuel-type-diesel",
    "gas": "fuel-type-gas"
})
df = pd.concat([df, fuel_dummies], axis=1)
df.drop("fuel-type", axis=1, inplace=True)

aspiration_dummies = pd.get_dummies(
    df["aspiration"]
).rename(columns={
    "std": "aspiration-std",
    "turbo": "aspiration-turbo"
})
df = pd.concat([df, aspiration_dummies], axis=1)
df.drop("aspiration", axis=1, inplace=True)
df.head()

## Save cleaned dataset

In [ ]:
output_path = (
    Path.cwd().parent.parent
    /"Data"
    /"Clean_Data"
    /"clean_auto_df.csv"
)
df.to_csv(output_path, index=False)
output_path